# rs_hash for row-streaming datasets

In [ ]:
# !git clone https://github.com/mayaawada/Test.git
# %cd Test/Row-streaming

#Or just download the files containing the data from https://github.com/mayaawada/Test.git

### Import required libraries

In [1]:
from sklearn.datasets import load_svmlight_file
from rs_hash import RSHash
import tqdm
from sklearn.metrics import average_precision_score, roc_auc_score
import time
import math


### Data loading

In [28]:
data = load_svmlight_file("data/Row-streaming/spam-sms")
X = data[0]
y = data[1]
X= X.todense()

### Run the algorithm

In [32]:
window_size = int(0.01*len(y))
k = 100
n_chains = 100
depth = 15

all_scores = []

start_time = time.time()


cf = RSHash(X.min(axis=0).A1, X.max(axis=0).A1) 

for i, sample in enumerate(tqdm.tqdm(X)):
  cf.learn_one(sample.A1)
  if i>=window_size:
    anomalyscore = -cf.score_one(sample.A1)
    all_scores.append(anomalyscore)

print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 5574/5574 [00:16<00:00, 346.61it/s]

--- 16.219448804855347 seconds ---


### Compute mean average precision and ROC metric

In [33]:
y_adjusted = y[window_size:window_size+len(all_scores)]

chunks = [all_scores[x:x+window_size] for x in range(0, len(all_scores), window_size)]
y_chunks = [y_adjusted[x:x+window_size] for x in range(0, len(y_adjusted), window_size)]

AP_window = []

for i in range(len(y_chunks)-1):
  score = average_precision_score(y_chunks[i], chunks[i])
  AP_window.append(score)

### Results

In [34]:
OAP = average_precision_score(y_adjusted, all_scores) 
MAP = sum(AP_window)/len(AP_window)
AUC = roc_auc_score(y_adjusted, all_scores)

print("XStream: OAP =", OAP,"\n\t",
      "MAP =", MAP, "\n\t", 
      "AUC =", AUC)

XStream: OAP = 0.2144878385382346 
	 MAP = 0.33296001711371126 
	 AUC = 0.6444245470825114
